# GREW-count (TD)

Sources : ce TD est largement repris d'une proposition de [Hee-Soo Choi](https://hschoi4.github.io/) et a été corrigé grâce à l'aide de [Bruno Guillaume](https://members.loria.fr/BGuillaume/).

La fonction request donne accès au serveur de GREW-count et permet de récupérer le nombre d’occurrences du pattern demandé. 

La fonction prend en paramètres :
- Une liste de corpus (à choisir)
- Une liste de patterns (à définir)


In [1]:
import requests
import json
from pprint import pprint

def request(corpora, patterns):
    response = requests.request(
        "POST",
        "http://grew.marceau-h.fr/",
        headers= {'Content-Type': 'application/x-www-form-urlencoded'},
        data={ 'corpora': json.dumps(corpora), 'requests': json.dumps(patterns) }
    )
    return response.text

On définit ci-dessous les corpus à prendre en compte et les motifs (patterns) à appliquer sur ces corpus. 

Dans l'exemple, on prend en compte un corpus de l'anglais (UD_English-EWT@2.11) et on cherche tous les verbes (résultat : 28059).

In [2]:
corpus = ["UD_French-GSD@2.12" ]

patterns = {'VERB': 'pattern {V [upos=VERB]}',
            'VERB-o': 'pattern {V [upos=VERB];'
                      'V -[obj]-> O}',
            'VERB-oi': 'pattern {V [upos=VERB];'
                       'V -[iobj]-> I}',
            }

print(request(corpus, patterns))

<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js">
<!--<![endif]-->

<head>
	<meta charset="utf-8">
	<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">

	<title>Grew-match</title>

	<meta name="description" content="">
	<meta name="viewport" content="width=device-width, initial-scale=1">

	<link rel="stylesheet" href="css/bootstrap.min.css">

	<link rel="stylesheet" href="vendor/bootflat.css">

	<script src="vendor/jquery-2.1.1.min.js"></script>
	<script src="https://cdn.jsdelivr.net/npm/vue/dist/vue.js"></script>

	<!-- tooltipter -->
	<link rel="stylesheet" href="vendor/tooltipter/tooltipster.css">
	<link rel="stylesheet" href="vendor/tooltipter/tooltipster-noir.css">
	<script src="vendor/tooltipter/jquery.tooltipster.min.js"></script>

	<link rel="style

### Exercice 1 : déterminez si l’adjectif s’accorde en nombre avec le nom pour l'anglais, le français, l'espagnol, le japonais et le chinois.

In [17]:
corpus = [
  "UD_English-PUD@2.11",
  "UD_French-PUD@2.11",
  "UD_Spanish-PUD@2.11",
  "UD_Japanese-PUD@2.11",
  "UD_Chinese-PUD@2.11",
]

patterns = {
    'Tot' : 'pattern {N -[amod]-> A;}',
    'ADJ N' : ' pattern {N [Number]; A [Number];N -[amod]-> A;}',

    'ADJ == N' : 'pattern {N -[amod]-> A; A.Number = N.Number;}',
    'ADJ != N' : 'pattern {N -[amod]-> A; A.Number <> N.Number;}',

}

x = [e.split("\t") for e in request(corpus, patterns).split("\n") if e]
entete = x[0]
for e in x[1:]:
    print(f"{entete[0]} : {e[0]} ")
    print(f"{entete[2]} : {e[2]} ")
    print(f"{entete[3]} : {e[3]} ")
    if int(e[3]):
        print(f"{entete[4]} : {e[4]} ({int(e[4])/int(e[3])*100:.2f}%) ")
        print(f"{entete[5]} : {e[5]} ({int(e[5])/int(e[3])*100:.2f}%) ")
    else:
        print("pas de nombres")
    print()

Dans les langues qui expriment les nombres des noms et adjectifs, les adjectifs et noms s'accordent dans la plupart des cas, un peu moins pour l'anglais

### Exercice 2 : pour les mêmes langues, déterminez si l’adjectif se trouve avant ou après le nom (ou les deux).

In [4]:
patterns = {
    'Tot' : 'pattern {N -[amod]-> A;}',
    'ADJ << N' : 'pattern {N -[amod]-> A; A << N;}',
    'ADJ >> N' : 'pattern {N -[amod]-> A; A >> N;}',

}

x = [e.split("\t") for e in request(corpus, patterns).split("\n") if e]
entete = x[0]

for e in x[1:]:
    print(f"{entete[0]} : {e[0]} ")
    print(f"{entete[2]} : {e[2]} ")
    if int(e[3]):
        print(f"{entete[3]} : {e[3]} ({int(e[3])/int(e[2])*100:.2f}%) ")
        print(f"{entete[4]} : {e[4]} ({int(e[4])/int(e[2])*100:.2f}%) ")
    else:
        print("pas de nombres")
    print()

### Exercice 3 : choisissez 3 langues que vous ne connaissez pas et déterminez leur type (SOV, VSO, OVS, etc).

In [5]:
corpus = [
  "UD_Chinese-PUD@2.11",
  "UD_Czech-PUD@2.11",
  "UD_Swedish-PUD@2.11",
]

patterns = {
    "Tot" : """pattern {
  Verbe [upos = VERB];
  Verbe -[nsubj]-> Sujet;
  Verbe -[obj]-> Objet;
    }""",

    "SVO" : """pattern {
  Verbe [upos = VERB];
  Verbe -[nsubj]-> Sujet;
  Verbe -[obj]-> Objet;
  Sujet << Verbe;
  Verbe << Objet;
    }""",

    "SOV" : """pattern {
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Sujet << Objet;
    Objet << Verbe;
    }""",

    "VSO" : """pattern {
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Verbe << Sujet;
    Sujet << Objet;
    }""",

    "VOS" : """pattern {
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Verbe << Objet;
    Objet << Sujet;
    }""",

    "OSV" : """pattern {
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Objet << Sujet;
    Sujet << Verbe;
    }""",
}

x = [e.split("\t") for e in request(corpus, patterns).split("\n") if e]
entete = x[0]

for e in x[1:]:
    print(f"{entete[0]} : {e[0]} ")
    print(f"{entete[2]} : {e[2]} ")
    if int(e[3]):
        print(f"{entete[3]} : {e[3]} ({int(e[3])/int(e[2])*100:.2f}%) ")
        print(f"{entete[4]} : {e[4]} ({int(e[4])/int(e[2])*100:.2f}%) ")
        print(f"{entete[5]} : {e[5]} ({int(e[5])/int(e[2])*100:.2f}%) ")
        print(f"{entete[6]} : {e[6]} ({int(e[6])/int(e[2])*100:.2f}%) ")
        print(f"{entete[7]} : {e[7]} ({int(e[7])/int(e[2])*100:.2f}%) ")
    else:
        print("pas de nombres")
    print()

### Exercice 4 : pour les mêmes langues, déterminez si l'ordre sujet-verbe change si le sujet est un nom commun ou un pronom

In [6]:
patterns = {
    "Nom": """pattern {
    Sujet [upos = NOUN];
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    }""",

    "SVO": """pattern {
    Sujet [upos = NOUN];
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Sujet << Verbe;
    Verbe << Objet;
    }""",

    "SOV": """pattern {
    Sujet [upos = NOUN];
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Sujet << Objet;
    Objet << Verbe;
    }""",

    "VSO": """pattern {
    Sujet [upos = NOUN];
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Verbe << Sujet;
    Sujet << Objet;
    }""",

    "VOS": """pattern {
    Sujet [upos = NOUN];
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Verbe << Objet;
    Objet << Sujet;
    }""",

    "OSV": """pattern {
    Sujet [upos = NOUN];
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Objet << Sujet;
    Sujet << Verbe;
    }""",
}

x = [e.split("\t") for e in request(corpus, patterns).split("\n") if e]
entete = x[0]

for e in x[1:]:
    print(f"{entete[0]} : {e[0]} ")
    print(f"{entete[2]} : {e[2]} ")
    if int(e[3]):
        print(f"{entete[3]} : {e[3]} ({int(e[3]) / int(e[2]) * 100:.2f}%) ")
        print(f"{entete[4]} : {e[4]} ({int(e[4]) / int(e[2]) * 100:.2f}%) ")
        print(f"{entete[5]} : {e[5]} ({int(e[5]) / int(e[2]) * 100:.2f}%) ")
        print(f"{entete[6]} : {e[6]} ({int(e[6]) / int(e[2]) * 100:.2f}%) ")
        print(f"{entete[7]} : {e[7]} ({int(e[7]) / int(e[2]) * 100:.2f}%) ")
    else:
        print("pas de nombres")
    print()


In [7]:
patterns = {
    "Propre": """pattern {
    Sujet [upos = PROPN];
  Verbe [upos = VERB];
  Verbe -[nsubj]-> Sujet;
  Verbe -[obj]-> Objet;
    }""",

    "SVO": """pattern {
    Sujet [upos = PROPN];
  Verbe [upos = VERB];
  Verbe -[nsubj]-> Sujet;
  Verbe -[obj]-> Objet;
  Sujet << Verbe;
  Verbe << Objet;
    }""",

    "SOV": """pattern {
    Sujet [upos = PROPN];
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Sujet << Objet;
    Objet << Verbe;
    }""",

    "VSO": """pattern {
    Sujet [upos = PROPN];
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Verbe << Sujet;
    Sujet << Objet;
    }""",

    "VOS": """pattern {
    Sujet [upos = PROPN];
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Verbe << Objet;
    Objet << Sujet;
    }""",

    "OSV": """pattern {
    Sujet [upos = PROPN];
    Verbe [upos = VERB];
    Verbe -[nsubj]-> Sujet;
    Verbe -[obj]-> Objet;
    Objet << Sujet;
    Sujet << Verbe;
    }""",
}

x = [e.split("\t") for e in request(corpus, patterns).split("\n") if e]
entete = x[0]

for e in x[1:]:
    print(f"{entete[0]} : {e[0]} ")
    print(f"{entete[2]} : {e[2]} ")
    if int(e[3]):
        print(f"{entete[3]} : {e[3]} ({int(e[3]) / int(e[2]) * 100:.2f}%) ")
        print(f"{entete[4]} : {e[4]} ({int(e[4]) / int(e[2]) * 100:.2f}%) ")
        print(f"{entete[5]} : {e[5]} ({int(e[5]) / int(e[2]) * 100:.2f}%) ")
        print(f"{entete[6]} : {e[6]} ({int(e[6]) / int(e[2]) * 100:.2f}%) ")
        print(f"{entete[7]} : {e[7]} ({int(e[7]) / int(e[2]) * 100:.2f}%) ")
    else:
        print("pas de nombres")
    print()


Chinois:
- Nom : 100% SVO
- Propre : 100% SVO

Tchèque :
- Nom :
    - 71,2% SVO
    - 8,7% SOV
    - 6,0% VSO
- Propre :
    - 63,7% SVO
    - 9,9% SOV
    - 8,8% VSO

Suédois:
- Nom :
    - 81.9% SVO
    - 14,4% VSO
    - 0% VOS
    - 0% OSV
- Propre :
    - 63,7% SVO
    - 22,2% VSO
    - 3,7% VOS
    - 3,7% OSV

### Exercice 5 : déterminez si le déterminant s’accorde en genre avec le nom pour l'anglais, le français, l'espagnol, le japonais et le chinois (ou d'autres langues...)

In [8]:
corpus = [
  "UD_English-PUD@2.11",
  "UD_French-PUD@2.11",
  "UD_Spanish-PUD@2.11",
  "UD_Japanese-PUD@2.11",
  "UD_Chinese-PUD@2.11",
]

patterns = {
    "Tot" : 'pattern { N [upos="NOUN"]; N -[det]-> Det;}',
    'Det N' : ' pattern {N [Gender]; Det [Gender];N [upos="NOUN"]; N -[det]-> Det;}',

    'Det == N' : 'pattern {N [upos="NOUN"]; N -[det]-> Det; Det.Gender = N.Gender;}',
    'Det != N' : 'pattern {N [upos="NOUN"]; N -[det]-> Det; Det.Gender <> N.Gender;}',
}

x = [e.split("\t") for e in request(corpus, patterns).split("\n") if e]
entete = x[0]
for e in x[1:]:
    print(f"{entete[0]} : {e[0]} ")
    print(f"{entete[2]} : {e[2]} ")
    print(f"{entete[3]} : {e[3]} ")
    if int(e[3]):
        print(f"{entete[4]} : {e[4]} ({int(e[4])/int(e[3])*100:.2f}%) ")
        print(f"{entete[5]} : {e[5]} ({int(e[5])/int(e[3])*100:.2f}%) ")
    else:
        print("pas de genres")
    print()

### Exercice 6 : déterminez les temps verbaux des verbes (présent, passé et non indiqué)


In [9]:
patterns = {
    "Tot" : 'pattern {V [upos="VERB"];}',

    'Tense' : 'pattern {V [Tense]; V [upos="VERB"];}',

    'Pres' : 'pattern {V [upos="VERB"]; V.Tense = Pres;}',
    'Past' : 'pattern {V [upos="VERB"]; V.Tense = Past;}',
}

x = [e.split("\t") for e in request(corpus, patterns).split("\n") if e]

entete = x[0]
for e in x[1:]:
    print(f"{entete[0]} : {e[0]} ")
    print(f"{entete[2]} : {e[2]} ")
    print(f"{entete[3]} : {e[3]} ")
    if int(e[3]):
        print(f"{entete[4]} : {e[4]} ({int(e[4])/int(e[3])*100:.2f}%) ")
        print(f"{entete[5]} : {e[5]} ({int(e[5])/int(e[3])*100:.2f}%) ")
    else:
        print("pas de tense")
    print()
